I. Visualization of the Data

In [1]:
############################Second Model 
import gensim
from gensim.test.utils import datapath
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from nltk.corpus import stopwords 
from gensim import models
import numpy as np

from pprint import pprint

import spacy

import pickle
import re
import pyLDAvis
#import pyLDAvis.gensim

import matplotlib.pyplot as plt
import pandas as pd
import re
from gensim.utils import simple_preprocess

C:\Users\csking\Anaconda3\envs\new_LDA\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\csking\Anaconda3\envs\new_LDA\lib\site-packages\setuptools\_distutils\version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\csking\Anaconda3\envs\new_LDA\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\csking\Anaconda3\envs\new_LDA\lib\site-packages\setuptools\_distutils\version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\csking\Anaconda3\envs\new_LDA\lib\site-packages\matplotlib\__init__.py:152: DeprecationWarning: distuti

In [4]:
df = pd.read_excel("Data/China_GlobalTimes_9APR2009-31DEC2022_articlesonly.xlsx")
df['Date'] = pd.to_datetime(df['article_date'])
df['Year'] = df['Date'].dt.year
df= df[df["article_text"].str.contains("Taiwan", na=False)]
#df = df.sample(frac=.25)
df.shape

(16231, 6)

In [5]:
stop_words = stopwords.words('english')
stop_words.extend(["aos", "au", "ao","ai", "monday", "tuesday", "xinhua", "wednesday", "thursday", "friday", "saturday", "sunday", 'said', 'The','countries', 'also', 'two', 'year', 'world', 'would', 'country', 'first', 'people', 'international', 'new', 'one', 'President', 'military', 'percent', 'government', 'told', 'time', 'years', 'South', 'global', 'last', 'including', 'could', 'meeting', 'Minister', 'made', 'political', 'relations', 'But', 'well', 'Times', 'since', 'Tuesday', 'Global', 'Monday', 'This', 'Beijing', 'three', 'many', 'region', 'according', 'Wednesday', 'States', 'talks', 'Thursday', 'Foreign', 'sanctions', 'long', 'like', 'take', 'major', 'Syrian', 'media', '000', 'billion', 'million', 'make', 'reported', 'Asia', 'crisis', 'still', 'World', 'high','support', 'second', 'statement', 'held', 'day', 'Friday', 'visit', 'work', 'International', 'power', 'North', 'Xinhua', 'situation', 'market', 'part', 'system', 'Photo', 'city', 'energy', 'national','set', 'deal', 'state', 'added', 'even', 'issues', 'called', 'important', 'may', 'leaders', 'Shanghai', 'president', "about", "others", 'saying', 'regional', 'policy', 'Sunday', 'West', 'based', 'issue', 'East', 'way', 'bilateral', 'forces', 'summit', 'However', 'efforts', 'Council', 'space', 'become', 'back', 'members', 'State', 'local', 'joint', 'gas', 'four', 'interests', 'among', 'around', 'next', 'growth', 'report', 'future', 'help', 'news', 'recent', 'top', 'official', 'five', 'Wang', 'They', 'group', 'ties', 'use', 'conflict', 'agreement', 'past', 'team', 'need', 'sides', 'expected', 'week', 'number', 'Ministry','According', 'Saturday', 'continue', 'former', 'level', 'officials', 'New', 'good', 'win'])
#stop_list = list(stop_words)

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

def process_data(df, col): 
    df['Text'] = df[col].map(lambda x: x.lower())
    data = df.Text.values.tolist()
    data_words = list(sent_to_words(data))
    return remove_stopwords(data_words)


In [6]:
# Create Dictionary
data_words = process_data(df, 'article_text')
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [15]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=20,
                   random_state=0,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)
lda_model.save('lda.model')

In [7]:
#Reload model
lda_model = models.LdaModel.load('lda.model')

In [8]:
# Print the Keyword
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
topic_dict = {}
topic_words = lda_model.show_topics(num_topics=20, num_words=3, log=False, formatted=False)
topic_words
#Create Topic_Dict
# for each in topic_words:
#     topic_num = each[0]
#     words = each[1]
#     topic_dict[topic_num] = '-'.join(str(e[0]) for e in words)
# topic_dict

[(0, [('china', 0.040746953), ('us', 0.013176929), ('however', 0.011407831)]),
 (1, [('song', 0.053051542), ('photo', 0.03787599), ('chinese', 0.033802655)]),
 (2, [('japan', 0.16348325), ('china', 0.084680855), ('japanese', 0.0538271)]),
 (3, [('kong', 0.21273091), ('hong', 0.21209997), ('pandemic', 0.060257446)]),
 (4,
  [('chinese', 0.032356177),
   ('photo', 0.023428384),
   ('companies', 0.016196059)]),
 (5, [('us', 0.17955424), ('china', 0.076301485), ('pla', 0.020658297)]),
 (6,
  [('epidemic', 0.011254874),
   ('training', 0.0111901825),
   ('security', 0.0110333245)]),
 (7,
  [('development', 0.022643195),
   ('chinese', 0.021820705),
   ('china', 0.019752555)]),
 (8, [('arab', 0.11578467), ('opposes', 0.068297185), ('qin', 0.06325271)]),
 (9,
  [('theater', 0.04019474), ('event', 0.022067836), ('cultural', 0.02199254)]),
 (10,
  [('china', 0.036058713), ('economic', 0.030866781), ('supply', 0.02896189)]),
 (11,
  [('australia', 0.4700945),
   ('australian', 0.1871392),
   ('m

In [126]:
doc_topics = lda_model.get_document_topics(corpus)

# Get the documents related to topic 3
related_docs = []
for doc_id, topics in enumerate(doc_topics):
    for topic, prob in topics:
        if topic == 6 and prob >= 0.5:
            related_docs.append(doc_id)
            break

In [127]:
# Convert tuple of tuples corpus into list of word count dictionaries
doc_list = [dict(corpus[i]) for i in range(len(corpus))]

# View the related documents
for doc_id in related_docs:
    print(doc_id, ":", " ".join(id2word[word_id] for word_id, count in doc_list[doc_id].items()))
    print ("-"*100)

50 : china chinese kong province taiwan th wen worked beijing council demand due financial hong mainland manager markets previous shanghai states strong total traditional united across chief favorable days regions arrested found hotel left man old pick police selling authorities compared dr guangzhou led research already decision early health included jiabao ongoing organization premier principle regarding business hunan seven agriculture began believe diseases food later leave sense third took woman central foreign representative worker came however rather taking medicine send sent vice bring company crew material period return see step li macao want disease lived northern students body border following general ship yuan capital close citizens end figure landed line protect responsible attention keep necessary provide say think cases established law policies southern care hands towards evening board cooperate depends aircraft placed port worldwide angeles confirmed import los ministry

In [128]:
for i in related_docs:
    print (i, df['article_headline'].iloc[i])
    print ('-'*100)

50 China adopts, defends strict quarantine amid flu threat
----------------------------------------------------------------------------------------------------
61 Mainland flu suspect comes from the US
----------------------------------------------------------------------------------------------------
67 Beijing reports confirmed case of A(H1N1) flu
----------------------------------------------------------------------------------------------------
74 Taiwan reports confirmed case of A(H1N1) influenza
----------------------------------------------------------------------------------------------------
79 New flu cases continue to rise in Japan with first one confirmed in Tokyo
----------------------------------------------------------------------------------------------------
80 First deadly flu case confirmed in Taiwan
----------------------------------------------------------------------------------------------------
82 New flu cases continue to rise in Japan
-------------------------

In [83]:
topic_id = [i for i in range(20)]
doc_nums = []
doc_topic_probs = lda_model.get_document_topics(corpus)
for i in topic_id:
    topic_words = lda_model.get_topic_terms(i)
    # Get the documents and their probabilities for topic 0
    #doc_topic_probs = lda_model.get_document_topics(corpus)
    # Count the number of contributing documents to topic 0
    num_docs = sum(1 for doc in doc_topic_probs if any(topic[0] == i for topic in doc))
    doc_nums.append(num_docs)

# Print the result
print (doc_nums)
#print("Number of contributing documents to topic {}: {}".format(topic_id, num_docs))

[16143, 7418, 6843, 5736, 15714, 15610, 15641, 14645, 1673, 6757, 9015, 749, 3440, 14882, 13783, 12391, 2575, 3513, 16194, 11609]


In [104]:
#Get the topic-term matrix from the model
topic_term_matrix = lda_model.get_topics()

# Calculate the linkage value between all pairwise topics
n_topics = len(lda_model.get_topics())
linkage = np.zeros((n_topics, n_topics))
for i in range(n_topics):
    for j in range(i+1, n_topics):
        pij = 0
        for doc in corpus:
            doc_topics = lda_model.get_document_topics(doc)
            if doc_topics:
                topic_i = next((prob for topic, prob in doc_topics if topic == i), 0)
                topic_j = next((prob for topic, prob in doc_topics if topic == j), 0)
                pij += topic_i * topic_j
        pij /= len(corpus)
        pi = topic_term_matrix[i].sum() / topic_term_matrix.sum()
        pj = topic_term_matrix[j].sum() / topic_term_matrix.sum()
        rij = np.log2(pij / (pi * pj))
        linkage[i,j] = rij

# Output linkage to a CSV file
df = pd.DataFrame(linkage)
df.to_csv('linkage.csv', index=False)

In [4]:
df_linkage = pd.read_csv('linkage.csv')
topic_dict = {0: 'china-us-however',
 1: 'song-photo-chinese',
 2: 'japan-china-japanese',
 3: 'kong-hong-pandemic',
 4: 'chinese-photo-companies',
 5: 'us-china-pla',
 6: 'epidemic-training-security',
 7: 'development-chinese-china',
 8: 'arab-opposes-qin',
 9: 'theater-event-cultural',
 10: 'china-economic-supply',
 11: 'australia-australian-myanmar',
 12: 'online-liu-yuan',
 13: 'taiwan-island-mainland',
 14: 'china-cooperation-russia',
 15: 'wang-party-cpc',
 16: 'tsai-chen-election',
 17: 'south-korea-asia',
 18: 'west-want-type',
 19: 'chinese-photo-university'}
# Get the number of topics
n_topics = len(df_linkage)

# Create an empty DataFrame to hold the table
df_table = pd.DataFrame(columns=['Node I', 'Node J', 'Linkage Value'])

# Create an empty DataFrame to hold the table
df_table = pd.DataFrame(columns=['Node I', 'Node J', 'Linkage Value'])

# Iterate over the upper triangle of the linkage matrix
for i in range(n_topics):
    for j in range(i+1, n_topics):
        # Add a row to the table with the node indices and linkage value
        df_new_row = pd.DataFrame({
            'Node I': [i],
            'Node J': [j],
            'Linkage Value': [df_linkage.iloc[i,j]]
        })
        df_table = pd.concat([df_table, df_new_row], ignore_index=True)

df_table = df_table.sort_values(by=['Linkage Value'], ascending=False)
df_table['Node I'] = df_table['Node I'].map(topic_dict)
df_table['Node J'] = df_table['Node J'].map(topic_dict)
df_table.head
# Output the table to a CSV file
df_table.to_csv('linkage_table.csv', index=False)

In [28]:
time_periods = {
    2009: 'Period 1',
    2010: 'Period 1',
    2011: 'Period 1',
    2012: 'Period 1', 
    2013: 'Period 1',
    2014: 'Period 2', 
    2015: 'Period 2',
    2016: 'Period 2', 
    2017: 'Period 2', 
    2018: 'Period 3', 
    2019: 'Period 3', 
    2020: 'Period 3',
    2021: 'Period 3', 
    2022: 'Period 3'
    
}
df['Time_Period'] = df['Year'].map(time_periods)

In [18]:
df_period1 = df[df['Time_Period']== 'Period 1']
df_period1.shape

(6300, 8)

In [19]:
# Create Dictionary
data_words = process_data(df_period1, 'article_text')
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

C:\Users\csking\AppData\Local\Temp\ipykernel_5324\1784516359.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df[col].map(lambda x: x.lower())


In [22]:
#Get the topic-term matrix from the model
topic_term_matrix = lda_model.get_topics()

# Calculate the linkage value between all pairwise topics
n_topics = len(lda_model.get_topics())
linkage = np.zeros((n_topics, n_topics))
for i in range(n_topics):
    for j in range(i+1, n_topics):
        pij = 0
        for doc in corpus:
            doc_topics = lda_model.get_document_topics(doc)
            if doc_topics:
                topic_i = next((prob for topic, prob in doc_topics if topic == i), 0)
                topic_j = next((prob for topic, prob in doc_topics if topic == j), 0)
                pij += topic_i * topic_j
        pij /= len(corpus)
        pi = topic_term_matrix[i].sum() / topic_term_matrix.sum()
        pj = topic_term_matrix[j].sum() / topic_term_matrix.sum()
        rij = np.log2(pij / (pi * pj))
        linkage[i,j] = rij

# Output linkage to a CSV file
df1 = pd.DataFrame(linkage)
df1.to_csv('linkage_period1.csv', index=False)

In [23]:
df_linkage = pd.read_csv('linkage_period1.csv')
topic_dict = {0: 'china-us-however',
 1: 'song-photo-chinese',
 2: 'japan-china-japanese',
 3: 'kong-hong-pandemic',
 4: 'chinese-photo-companies',
 5: 'us-china-pla',
 6: 'epidemic-training-security',
 7: 'development-chinese-china',
 8: 'arab-opposes-qin',
 9: 'theater-event-cultural',
 10: 'china-economic-supply',
 11: 'australia-australian-myanmar',
 12: 'online-liu-yuan',
 13: 'taiwan-island-mainland',
 14: 'china-cooperation-russia',
 15: 'wang-party-cpc',
 16: 'tsai-chen-election',
 17: 'south-korea-asia',
 18: 'west-want-type',
 19: 'chinese-photo-university'}
# Get the number of topics
n_topics = len(df_linkage)

# Create an empty DataFrame to hold the table
df_table = pd.DataFrame(columns=['Node I', 'Node J', 'Linkage Value'])

# Create an empty DataFrame to hold the table
df_table = pd.DataFrame(columns=['Node I', 'Node J', 'Linkage Value'])

# Iterate over the upper triangle of the linkage matrix
for i in range(n_topics):
    for j in range(i+1, n_topics):
        # Add a row to the table with the node indices and linkage value
        df_new_row = pd.DataFrame({
            'Node I': [i],
            'Node J': [j],
            'Linkage Value': [df_linkage.iloc[i,j]]
        })
        df_table = pd.concat([df_table, df_new_row], ignore_index=True)

df_table = df_table.sort_values(by=['Linkage Value'], ascending=False)
df_table['Node I'] = df_table['Node I'].map(topic_dict)
df_table['Node J'] = df_table['Node J'].map(topic_dict)
df_table.head
# Output the table to a CSV file
df_table.to_csv('linkage_table_period1.csv', index=False)

In [29]:
df_period2 = df[df['Time_Period']== 'Period 2']
df_period2.shape

(4351, 7)

In [30]:
def get_corpus(df):
    # Create Dictionary
    data_words = process_data(df, 'article_text')
    id2word = corpora.Dictionary(data_words)

    # Create Corpus
    texts = data_words

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    return corpus

In [31]:
corpus = get_corpus(df_period2)

C:\Users\csking\AppData\Local\Temp\ipykernel_5324\1784516359.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df[col].map(lambda x: x.lower())


In [33]:
def computer_topic_term_matrix(model, corpus, title):
    #Get the topic-term matrix from the model
    topic_term_matrix = model.get_topics()

    # Calculate the linkage value between all pairwise topics
    n_topics = len(model.get_topics())
    linkage = np.zeros((n_topics, n_topics))
    for i in range(n_topics):
        for j in range(i+1, n_topics):
            pij = 0
            for doc in corpus:
                doc_topics = lda_model.get_document_topics(doc)
                if doc_topics:
                    topic_i = next((prob for topic, prob in doc_topics if topic == i), 0)
                    topic_j = next((prob for topic, prob in doc_topics if topic == j), 0)
                    pij += topic_i * topic_j
            pij /= len(corpus)
            pi = topic_term_matrix[i].sum() / topic_term_matrix.sum()
            pj = topic_term_matrix[j].sum() / topic_term_matrix.sum()
            rij = np.log2(pij / (pi * pj))
            linkage[i,j] = rij

    # Output linkage to a CSV file
    df1 = pd.DataFrame(linkage)
    df1.to_csv('Linkage Models/{}.csv'.format(title), index=False)

In [34]:
computer_topic_term_matrix(lda_model, corpus, "linkage_period2.csv")

In [35]:
topic_dict = {0: 'china-us-however',
     1: 'song-photo-chinese',
     2: 'japan-china-japanese',
     3: 'kong-hong-pandemic',
     4: 'chinese-photo-companies',
     5: 'us-china-pla',
     6: 'epidemic-training-security',
     7: 'development-chinese-china',
     8: 'arab-opposes-qin',
     9: 'theater-event-cultural',
     10: 'china-economic-supply',
     11: 'australia-australian-myanmar',
     12: 'online-liu-yuan',
     13: 'taiwan-island-mainland',
     14: 'china-cooperation-russia',
     15: 'wang-party-cpc',
     16: 'tsai-chen-election',
     17: 'south-korea-asia',
     18: 'west-want-type',
     19: 'chinese-photo-university'}

In [57]:
def convert_to_adjacency_matrix(csv_path, topic_dictionary, title):
    df_linkage = pd.read_csv("Linkage Models/{}".format(csv_path))
    
    # Get the number of topics
    n_topics = len(df_linkage)

    # Create an empty DataFrame to hold the table
    df_table = pd.DataFrame(columns=['Node I', 'Node J', 'Linkage Value'])

    # Iterate over the upper triangle of the linkage matrix
    for i in range(n_topics):
        for j in range(i+1, n_topics):
            # Add a row to the table with the node indices and linkage value
            df_new_row = pd.DataFrame({
                'Node I': [i],
                'Node J': [j],
                'Linkage Value': [df_linkage.iloc[i,j]]
            })
            df_table = pd.concat([df_table, df_new_row], ignore_index=True)

    df_table = df_table.sort_values(by=['Linkage Value'], ascending=False)
    #df_table['Node I'] = df_table['Node I'].map(topic_dictionary)
    #df_table['Node J'] = df_table['Node J'].map(topic_dictionary)
    df_table.head
    # Output the table to a CSV file
    df_table.to_csv('Linkage Models/{}'.format(title), index=False)

In [45]:
convert_to_adjacency_matrix("linkage_period2.csv", topic_dict, 'linkage_table_period2.csv')

In [46]:
df_period3 = df[df['Time_Period']== 'Period 3']
df_period3.shape

(5580, 7)

In [47]:
corpus = get_corpus(df_period3)

C:\Users\csking\AppData\Local\Temp\ipykernel_5324\1784516359.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df[col].map(lambda x: x.lower())


In [48]:
computer_topic_term_matrix(lda_model, corpus, "linkage_period3.csv")

In [52]:
convert_to_adjacency_matrix("linkage_period3.csv", topic_dict, 'linkage_table_period3.csv')

In [58]:
convert_to_adjacency_matrix("linkage.csv", topic_dict, 'linkage_table_string.csv')

In [59]:
convert_to_adjacency_matrix("linkage_period1.csv", topic_dict, 'linkage_table_period1_string.csv')

In [60]:
convert_to_adjacency_matrix("linkage_period2.csv", topic_dict, 'linkage_table_period2_string.csv')

In [61]:
convert_to_adjacency_matrix("linkage_period3.csv", topic_dict, 'linkage_table_period3_string.csv')